# maude-magic

> Execute a maude`s session in Jupyter Lab

**Refs:**
- [pexpect](https://pexpect.readthedocs.io/en/stable/)
- [nbdev](https://nbdev.fast.ai/tutorials/tutorial.html)
- [IPython magics](https://ipython.readthedocs.io/en/stable/config/custommagics.html)
- [Export examples](https://nbdev1.fast.ai/export.html#Examples-of-export)
- See persistent Shells.ipynb  

## The Maude Interpreter class

In [1]:
#| default_exp maude-magic

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| hide  
import os
import pexpect
import time
import re

In [4]:
#| export
timeout = 3


class TimeoutException(Exception):
    """Exception raised if time-out."""

    def __init__(self):
        super().__init__("Timeout exception.")
        self.error_code = 1

    def __str__(self):
        return f"{self.message} (Error Code: {self.error_code})"    


class MaudeInterpreter:
    """Controls maude execution, executing commands and print responses.
       Preserve sessions between different cell executions."""
    
    
    def __init__(self,debug=False,timeout=timeout):
        """ Init variables and spawns maude. """
        
        # Regular expressions to scan outputs
        __result_pair_RE = r"result *(\w+): *(\w+) *\r"
    
        self.debug,self.timeout = debug,timeout
        # Execute maude in environment "env". Add current working directory to MAUDE_LIB
        env = dict(os.environ)
        if not 'MAUDE_LIB' in env: raise(Exception('MAUDE_LIB environmet variable not found'))
        env['MAUDE_LIB'] += ':' + os.getcwd()
        if self.debug: 
            print(f"MAUDE_LIB={env['MAUDE_LIB']}")
            
        # Compile regular expressions to Regular Expression Objects
        self.__result_pair_REO = re.compile(__result_pair_RE,re.MULTILINE)
        
        # The expawned process:
        self.sh = pexpect.spawn('maude --no-advise', encoding='utf-8', env=env)
        self.sh.expect('Maude> ')
        self._sync()    

    def _sync(self):
        """ We syncrhonize with maude shell requesting a pwd.""" 
        self.sh.sendline('pwd')
        self.sh.expect('pwd.*Maude> ')
            
    def __del__(self):
        if self.debug: print('Destroying Object')
        self.__call__('quit .')

    def _filter_response(self,response:str)->tuple:
        maude_cmd,blank,maude_args = response.partition(' ')
        print("__filter_response:maude_cmd,blank,maude_args=",maude_cmd,blank,maude_args)
        # print('cmd=',maude_cmd,'blank=',blank,'args=',maude_args)
        if maude_cmd in ["load","show"]:
            return (maude_cmd,maude_args)
        match = self.__result_pair_REO.search(response)
        # print('match=',match)
        if match:
            return  (match.group(1),match.group(2))
        return None    
            
    # Cmmand processing
    def __call__(self,command,timeout=timeout):
        """ Adds '\n' to command if it don't terminate with it. """
        # Don't terminate session by command
        if self.debug: print(f"Original Command = {repr(command)}")
        if command == 'quit .': return ''
        # strip command before send
        print(f"<--{repr(command.strip())}")
        if command[-1] != '\n' : command += '\n'
        print(f"Sent Command = {repr(command)}")
        self.sh.send(command)
        self._sync()
        print(f"-->{repr(self.sh.before)+repr(self.sh.after)}")
        response = self.sh.before
        # filter response
        return self._filter_response(response)
        

In [5]:
def assert_print(result:tuple,type_=None,value_=None):
    # print('result=',result)
    if not result : return 
    
    if result[1] in ["load","show"]:
        print(result[2])
        return 
    if type_ or value_:
        if type_ :  assert result[1] == type_
        if value_:  assert result[2] == value_    
    else:
        print(result)
        

Creating maude interpreter:

In [6]:
#| hide
maude=MaudeInterpreter(debug=False)

Load a maude module. Show it and make some reduction:

In [7]:
result= maude('load SIMPLE-NAT .','load')
assert_print(result)

<--'load SIMPLE-NAT .'
Sent Command = 'load SIMPLE-NAT .\n'
-->'load SIMPLE-NAT .\n\r\rMaude> ''pwd\n\r/home/usuario\r\n\rMaude> '
__filter_response:maude_cmd,blank,maude_args= load   SIMPLE-NAT .
Maude> 
('load', 'SIMPLE-NAT .\n\r\rMaude> ')


In [8]:
#| hide
result = maude('red s s zero .')
assert_print(result,'Nat','s s zero')

<--'red s s zero .'
Sent Command = 'red s s zero .\n'
-->'red s s zero .\n\rreduce in SIMPLE-NAT : s s zero .\r\nrewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)\r\nresult Nat: s s zero\r\n\rMaude> ''pwd\n\r/home/usuario\r\n\rMaude> '
__filter_response:maude_cmd,blank,maude_args= red   s s zero .
reduce in SIMPLE-NAT : s s zero .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Nat: s s zero
Maude> 


Skip `quit` command:

In [9]:
#| hide
maude('quit .')

''

Closing maude session on object destroy:

In [10]:
#| hide
del maude 

## The maude Magic Class

In [11]:
#| hide
from IPython.core.magic import (Magics, magics_class, line_magic,
                                cell_magic, line_cell_magic)

In [12]:
#| export
# This code can be put in any Python module, it does not require IPython
# itself to be running already.  It only creates the magics subclass but
# doesn't instantiate it yet.
# from __future__ import print_function
from IPython.core.magic import (Magics, magics_class, line_magic,
                                cell_magic, line_cell_magic)

# The class MUST call this class decorator at creation time
@magics_class
class MaudeMagics(Magics):
    """Adapts Maude Shell to a IPython Magic class.Uses a owned Maude Shell.
       Cell magics are used to execute maude commands.
       Line magics are used for command line options."""  
    
    __maude_shell = None 
    
    def __init__(self,shell):
        # Create the owned Maude Shell instance an pass it as shell
        __maude_shell = MaudeInterpreter(debug=False)
        super(MaudeMagics,self).__init__(__maude_shell) 
        # print("On Construntor:"+str(type(self.shell)))
        self.line_counter=0
    
    def get_maude_shell(self):
        return __maude_shell
        
    def prepare_request(self,cell_contents:str)->str:
        """ - Removes leading and trailing empty lines from cell
            - Removes leading and trailing empty lines from each line
            - Terminates each line with \r\n
            - Count efective lines sent to maude shell.
        """
        # print(f"cell_contents at maude()={cell_contents}")
        cell_contents=cell_contents.strip()
        self.before_count =self.line_counter
        request = ""
        for cell_line in cell_contents.split('\n'):
            request+=cell_line.strip()+'\n'
            self.line_counter+=1
        # Maude shell will add trailing \n
        return request
    
    __prepare_request=prepare_request 

    def prepare_response(self,shell_response:tuple)->str:
        """ Ads the count of sent lines at the header of respone,
            to ease sintax error location.
        """    
        response =  f"{self.before_count} (lines sent before.)\n"
        response += ("--------------------------------------------\n")
        for line in shell_response.split('\n'):
            line =  line.strip()
            if line[0]=='>':
                self.before_count += 1
                line = f"{self.before_count:3} {line}"
            response += line + '\n'
        return response
    
    __prepare_response = prepare_response     
           
    @line_cell_magic
    def maude(self, line, cell=None):
        if cell is None:
            print("MaudeInterpreter:","shell line=",line)
            result = self.shell(self.prepare_request(line))
            print("MaudeInterpreter:",'result=',result)
            return result
        else:
            # print(f"cell at maude()={cell}")
            # TODO: migrar el cálculo del nº de línea a MaudeInterpreter
            # print(self.prepare_response(self.shell(self.prepare_request(cell))))
            print(self.shell(self.prepare_request(cell)))

# In order to actually use these magics, you must register them with a
# running IPython.

def load_ipython_extension(ipython):
    """
    Any module file that define a function named `load_ipython_extension`
    can be loaded via `%load_ext module.path` or be configured to be
    autoloaded by IPython at startup time.
    """
    # You can register the class itself without instantiating it.  IPython will
    # call the default constructor on it.
    ipython.register_magics(MaudeMagics)

Manually executing 'load_ipython_extension' for test purposes:

In [13]:
#| hide
load_ipython_extension(get_ipython())

Now, MaudeMagic uses an owned MaudeInterpreter to run maude commands:

In [14]:
#| hide
result = %maude load SIMPLE-NAT .
print(result)
    
    

MaudeInterpreter: shell line= load SIMPLE-NAT .
<--'load SIMPLE-NAT .'
Sent Command = 'load SIMPLE-NAT .\n'
-->'load SIMPLE-NAT .\n\r\rMaude> ''pwd\n\r/home/usuario\r\n\rMaude> '
__filter_response:maude_cmd,blank,maude_args= load   SIMPLE-NAT .
Maude> 
MaudeInterpreter: result= ('load', 'SIMPLE-NAT .\n\r\rMaude> ')
('load', 'SIMPLE-NAT .\n\r\rMaude> ')


In [15]:
result = %maude show module .
assert_print(result)  

MaudeInterpreter: shell line= show module .
<--'show module .'
Sent Command = 'show module .\n'
-->'show module .\n\rfmod SIMPLE-NAT is\r\n  sort Nat .\r\n  op zero : -> Nat .\r\n  op s_ : Nat -> Nat .\r\n  op _+_ : Nat Nat -> Nat .\r\n  vars M N : Nat .\r\n  eq zero + N = N .\r\n  eq s N + M = s (N + M) .\r\nendfm\r\n\rMaude> ''pwd\n\r/home/usuario\r\n\rMaude> '
__filter_response:maude_cmd,blank,maude_args= show   module .
fmod SIMPLE-NAT is
  sort Nat .
  op zero : -> Nat .
  op s_ : Nat -> Nat .
  op _+_ : Nat Nat -> Nat .
  vars M N : Nat .
  eq zero + N = N .
  eq s N + M = s (N + M) .
endfm
Maude> 
MaudeInterpreter: result= ('show', 'module .\n\rfmod SIMPLE-NAT is\r\n  sort Nat .\r\n  op zero : -> Nat .\r\n  op s_ : Nat -> Nat .\r\n  op _+_ : Nat Nat -> Nat .\r\n  vars M N : Nat .\r\n  eq zero + N = N .\r\n  eq s N + M = s (N + M) .\r\nendfm\r\n\rMaude> ')
('show', 'module .\n\rfmod SIMPLE-NAT is\r\n  sort Nat .\r\n  op zero : -> Nat .\r\n  op s_ : Nat -> Nat .\r\n  op _+_ : Nat 

In [16]:
result = %maude red s s 0 .
assert_print(result) 

MaudeInterpreter: shell line= red s s 0 .
<--'red s s 0 .'
Sent Command = 'red s s 0 .\n'
-->"red s s 0 .\n\r\x1bWarning: \x1b<standard input>, line 6: didn't expect token \x1b0\x1b:\r\ns s 0 <---*HERE*\r\n\x1bWarning: \x1b<standard input>, line 6: no parse for term.\r\n\rMaude> "'pwd\n\r/home/usuario\r\n\rMaude> '
__filter_response:maude_cmd,blank,maude_args= red   s s 0 .
s s 0 <---*HERE*
Maude> 
MaudeInterpreter: result= None


## Use cases as test

Utility function that test than a provided maude command is of sort Bool and value = True .

### RDF Dataset

fth RDF-DATASET is 

    sort RDF-Dataset .

    op <_,_> : UnnamedGraph NamedGraph* -> RDF-Dataset [ctor] .    
    op NamedGraph : ResourceIdentifier Graph odeIdentifier    
        
        
        
        sort RDF-Document .

    op serializes : RDF-Dataset -> JSON-LD-Document .
    op serializes : Property -> IRI . 

    
    sorts NamedGraph NamedGraph+ NamedGraph* .

    
        


    op serializes : Property -> ResourceIdentifier .     
    
    op namedGraph : NodeIdentifier Graph -> 
    op _-[_]->_ : SourceNode IRI Node -> Arc [ctor] . 


    IRI BlankNodeId < SourceNodeId < NodeId Literal < NodeId .   
        

### JSON Syntax

[JSON-LD 1.1](https://www.w3.org/TR/json-ld11/#uses-of-json-objects)

In [17]:
%%maude 
fmod JSON-SORTS is
    sort Json 
    sorts  Object Array .
    subsorts Object Array < Json .    
    sort Entry .    
    op null : -> Json [ctor] .    
endfm         
  
view Json  from TRIV to JSON-SORTS is sort Elt to Json . endv
view Entry from TRIV to JSON-SORTS is sort Elt to Entry . endv

fmod JSON-ARRAY is
    protecting LIST{Json} * ( op (__) to (_,_) ) .
    op [_] : List{Json} -> Array .
    eq [nil] = null .
    op [] : -> Array [ctor] .
    eq [] = null .    
endfm

fmod JSON-OBJECT is
    protecting SET{Entry} .
    op {_} : Set{Entry} -> Object .
    eq {empty} = null .    
    
    sorts Key Value .
    op _:_ : Key Value  -> Entry [ctor] .
endfm        
             
fmod JSON is
    extending JSON-SORTS .
    protecting JSON-ARRAY .
    protecting JSON-OBJECT .        
    protecting STRING .

    subsort String  < Key Json .
    subsort Json < Value .
        
endfm 

<--'fmod JSON-SORTS is\nsort Json\nsorts  Object Array .\nsubsorts Object Array < Json .\nsort Entry .\nop null : -> Json [ctor] .\nendfm\n\nview Json  from TRIV to JSON-SORTS is sort Elt to Json . endv\nview Entry from TRIV to JSON-SORTS is sort Elt to Entry . endv\n\nfmod JSON-ARRAY is\nprotecting LIST{Json} * ( op (__) to (_,_) ) .\nop [_] : List{Json} -> Array .\neq [nil] = null .\nop [] : -> Array [ctor] .\neq [] = null .\nendfm\n\nfmod JSON-OBJECT is\nprotecting SET{Entry} .\nop {_} : Set{Entry} -> Object .\neq {empty} = null .\n\nsorts Key Value .\nop _:_ : Key Value  -> Entry [ctor] .\nendfm\n\nfmod JSON is\nextending JSON-SORTS .\nprotecting JSON-ARRAY .\nprotecting JSON-OBJECT .\nprotecting STRING .\n\nsubsort String  < Key Json .\nsubsort Json < Value .\n\nendfm'
Sent Command = 'fmod JSON-SORTS is\nsort Json\nsorts  Object Array .\nsubsorts Object Array < Json .\nsort Entry .\nop null : -> Json [ctor] .\nendfm\n\nview Json  from TRIV to JSON-SORTS is sort Elt to Json . endv\

In [19]:
result = %maude red "Juan" : "Perico" .
assert_print(result,'Entry')    


MaudeInterpreter: shell line= red "Juan" : "Perico" .
<--'red "Juan" : "Perico" .'
Sent Command = 'red "Juan" : "Perico" .\n'
-->'red "Juan" : "Perico" .\n\rreduce in JSON : "Juan" : "Perico" .\r\nrewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)\r\nresult Entry: "Juan" : "Perico"\r\n\rMaude> ''pwd\n\r/home/usuario\r\n\rMaude> '
__filter_response:maude_cmd,blank,maude_args= red   "Juan" : "Perico" .
reduce in JSON : "Juan" : "Perico" .
rewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)
result Entry: "Juan" : "Perico"
Maude> 
MaudeInterpreter: result= None


In [20]:
result = %maude red \
{\
  "name" : "Manu Sporny" ,\
  "homepage" : "http://manu.sporny.org/" ,\
  "image" : "http://manu.sporny.org/images/manu.png"\
} .
assert_print(result,"Object")    

MaudeInterpreter: shell line= red  {   "name" : "Manu Sporny" ,   "homepage" : "http://manu.sporny.org/" ,   "image" : "http://manu.sporny.org/images/manu.png" } .
<--'red  {   "name" : "Manu Sporny" ,   "homepage" : "http://manu.sporny.org/" ,   "image" : "http://manu.sporny.org/images/manu.png" } .'
Sent Command = 'red  {   "name" : "Manu Sporny" ,   "homepage" : "http://manu.sporny.org/" ,   "image" : "http://manu.sporny.org/images/manu.png" } .\n'
-->'red  {   "name" : "Manu Sporny" ,   "homepage" : "http://manu.sporny.org/ \x08" ,   "image" : "http://manu.sporny.org/images/manu.png" } .\n\rreduce in JSON : {("homepage" : "http://manu.sporny.org/"\x1b,\x1b "image" :\r\n    "http://manu.sporny.org/images/manu.png")\x1b,\x1b "name" : "Manu Sporny"}\x1b .\r\nrewrites: 0 in 0ms cpu (0ms real) (~ rewrites/second)\r\nresult Object: {"homepage" : "http://manu.sporny.org/"\x1b,\x1b "image" :\r\n    "http://manu.sporny.org/images/manu.png"\x1b,\x1b "name" : "Manu Sporny"}\x1b\r\n\rMaude> ''

In [ ]:
result = %maude red ["Juan","Perico","Andres"] .
print(result)    
assert: 'Array: ["Juan", "Perico", "Andres"]' in result


In [ ]:
result = %maude red ["Juan",["Perico","Andres"]] .
print(result)    
assert 'Array: ["Juan", ["Perico", "Andres"]]' in result 

In [ ]:
result = %maude red ["Juan", {"Perico" : "Andres"} ] .
print(result)    
assert 'Array: ["Juan", {"Perico" : "Andres"}]' in result 

In [ ]:
result = %maude red {"Juan" : {"Perico" : "Andres"} } .
print(result)    
assert 'Object: {"Juan" : {"Perico" : "Andres"}}' in result 

In [ ]:
result = %maude red {"Juan" : ["Perico", "Andres"]} .
print(result)    
assert 'Object: {"Juan" : ["Perico", "Andres"]}' in result 

In [ ]:
result = %maude red \
{\
  "name" : "Manu Sporny",\
  "homepage" : "http://manu.sporny.org/",\
  "image" : "http://manu.sporny.org/images/manu.png"\
} .
print(result)
assert 'Object:' in result

In [ ]:
result = %maude red \
{\
  "@context" : {\
    "name" : "http://schema.org/name",\
    "image" : {\
      "@id" : "http://schema.org/image",\
      "@type" : "@id"\
    },\
    "homepage" : {\
      "@id" : "http://schema.org/url",\
      "@type" : "@id"\
    }\
  }\
} .
print(result)
assert 'Object:' in result

### IRIS
[JSON-LD 1.1](https://www.w3.org/TR/json-ld11/#terms)

In [ ]:
%%maude
fmod IRI is
protecting STRING .
sort IRI .
subsort IRI < String .
cmb S:String : IRI if find(S:String,":",0) :: NzNat .
endfm    


In [ ]:
result = %maude red "Http://perico" .
assert 'IRI: "Http://perico"' in result   

### Keywords

In [ ]:
%%maude
fmod KEYWORD is
    protecting STRING .
    sort Keyword .
    subsort Keyword < String .
    mb "@context"  : Keyword .
    mb "@id"       : Keyword .
    mb "@context"  : Keyword .
    mb "@id"       : Keyword .
    mb "@included" : Keyword .
    mb "@graph"    : Keyword .
    mb "@nest"     : Keyword .
    mb "@type"     : Keyword .
    mb "@reverse"  : Keyword .
    mb "@index"    : Keyword .
endfm    


In [ ]:
result = %maude red "@context" . 
print(result)
assert 'Keyword: "@context"' in result     

### Context

In [ ]:
%%maude 
fmod CONTEXT is
    protecting IRI .
    extending JSON *( sort Object to Json-Object ) .
        
    sorts TermDef Context .
    subsort TermDef < Entry .
    subsort Context < Json-Object .
        
    sorts ContextKey ContextValue .
    subsort String < ContextKey < Key .
    subsorts IRI Json-Object < ContextValue < Value .

    mb (K:ContextKey : V:ContextValue) : TermDef .
    var ... : Set{Entry} . 
    cmb { T:TermDef , ...} : Context if {...} :: Context .
    mb null : Context .
        
endfm  


In [ ]:
#--- Example 4: Context for the sample document in the previous section
result = %maude red\
    "name" : "http://schema.org/name"\
    ***(This means that 'name' is shorthand for 'http://schema.org/name')\
.
print(result)
assert('result TermDef:' in result) 

In [ ]:
#--- Example 4: Context for the sample document in the previous section
result = %maude red\
{\
    "name" : "http://schema.org/name"\
     ***(This means that 'name' is shorthand for 'http://schema.org/name')\
} .
print(result)
assert('result Context:' in result) 

In [ ]:
#--- Example 4: Context for the sample document in the previous section
result = %maude red { empty } .
print(result)
assert('result Context:' in result) 

In [ ]:
#--- Example 4: Context for the sample document in the previous section
result = %maude red\
{\
    "@context" : {\
        "name" : "http://schema.org/name",\
        ***(  This means that 'name' is shorthand for 'http://schema.org/name')\
        "image" : {\
            "@id" : "http://schema.org/image",\
            ***(  This means that 'image' is shorthand for 'http://schema.org/image')\
            "@type" : "@id"\
            ***( This means that a string value associated with 'image')\
            ***( should be interpreted as an identifier that is an IRI)\
            },\
        "homepage" : {\
            "@id" : "http://schema.org/url",\
            ***( This means that 'homepage' is shorthand for 'http://schema.org/url')\
            "@type" : "@id"\
            ***(  This means that a string value associated with 'homepage')\
            ***(  should be interpreted as an identifier that is an IRI )\
            }\
        }\
} .
print(result)
assert('result Context:' in result)

### Identifiers 
[JSON-LD 1.1 / Node Objects](https://www.w3.org/TR/json-ld11/#node-objects)

In [ ]:
%%maude
fmod IDENTIFIER is
    protecting STRING .
    protecting IRI .
    sorts BNI Compact-IRI Identifier .
    subsorts IRI Compact-IRI BNI < Identifier < String .
    var S : String .    
    cmb S : IRI  if find(S,":",0) :: Nat .    
endfm    
        

In [ ]:
result = %maude red "friends:perico" .
print(result)
assert("result IRI:" in result)

In [ ]:
%%maude 
fmod NODE-OBJECT-DECL is 
sort Node-Object .
endfm     

### Graph 

In [ ]:
%%maude
fmod GRAPH is 
    extending NODE-OBJECT-DECL .
    extending JSON *( sort Object to Json-Object ) .

    sorts Graph Named-Graph .
    subsorts Node-Object Named-Graph < Graph < Array .    
    subsort  Named-Graph < Json-Object .
    
    mb null : Graph .
    vars NObj NObj' NObj'' : Node-Object . var ... : List{Json} .  
    var V : Value .  var .... : Set{Entry} . 
    var Ng : Named-Graph .
     mb { "@id" : V, .... } : Named-Graph .
    cmb [ NObj , ... ] : Graph if  [ ... ] :: Graph .
endfm     

In [ ]:
result = %maude red "@id" : "http://www.schema.org/graph" .
print(result)   
assert('result Entry:' in result)

In [ ]:
result = %maude red .... :: Set{Entry} .
print(result)
assert('result Bool: true' in result)

In [ ]:
result = %maude red Ng :: Json .
print(result)
assert('result Bool: true' in result)

In [ ]:
result = %maude red { "@id" : "http://www.schema.org/graph", .... } .
print(result)
assert('result Named-Graph:' in result)                      

In [ ]:
result = %maude red [] . 
print(result)
assert('Graph: null' in result) 
result = %maude red [NObj] . 
print(result)
assert('result Graph: [NObj]' in result) 
result = %maude red [NObj, NObj' , NObj'' ] . 
print(result)
assert("result Graph: [NObj, NObj', NObj'']" in result) 

### Reverse map
[JSON-LD 1.1 / Reverse Properties](https://www.w3.org/TR/json-ld11/#reverse-properties)  
[JSON-LD 1.1 / Node Object](https://www.w3.org/TR/json-ld11/#node-objects)

In [ ]:
%%maude
fmod REVERSE-MAP is 
    protecting IDENTIFIER .
    extending NODE-OBJECT-DECL .
    extending JSON *( sort Object to Map ) .

    ***(If the node object contains the @reverse key,
        its value (ReverseKeyValue) MUST be a map containing entries (ReverseProperty) 
        representing reverse properties)    
    
    sorts ReverseValue .
    subsort ReverseValue < Map . 

     mb { RP:ReverseProperty } : ReverseValue .
    cmb { RP:ReverseProperty,Rest:NeSet{Entry} } : ReverseValue if { Rest:NeSet{Entry}} : ReverseValue .  

    sort ReverseProperty .
    subsort ReverseProperty < Entry .    
    sorts ReversePropertyValue .

        
    ***(Each value of such a reverse property (ReverseProperty) MUST be an IRI reference, a compact IRI, a blank node identifier, 
        a node object or an array containing a combination of these.)
    
    mb (ReversePropertyKey:IRI : V:ReversePropertyValue) : ReverseProperty .

    subsorts Identifier Node-Object ReversePropertyValueArray  < ReversePropertyValue . 
    
    sort ReversePropertyValueArray .
    
    subsort ReversePropertyValueArray  < Array .

    var RPV : ReversePropertyValue . var .... : NeList{Json} . 
     mb [ RPV ] : ReversePropertyValueArray .       
    cmb [ RPV , .... ] : ReversePropertyValueArray if [ .... ] :: ReversePropertyValueArray .   
        
endfm     

In [ ]:
result = %maude red  "http://perico.com"  . 
print(result)
assert(True) 
result = %maude red  "http://perico.com" :: ReversePropertyValue . 
print(result)
assert(True) 
result = %maude red  { "http://perico.com" : "http://perico.com" } :: ReverseValue . 
print(result)
assert('result Bool: true' in result) 
result = %maude red  { "http://perico.com" : "http://perico.com", \
                       "http://perico.com" : "http://perico.com" } :: ReverseValue . 
print(result)
assert('result Bool: true' in result) 

result = %maude red  [ "http://perico.com" ] . 
print(result)

result = %maude red  [ "http://perico.com" , "http://perico.com" ] . 
print(result)
#assert('result Bool: true' in result) 



#assert('result Bool: true' in result) 


In [ ]:
result = %maude red \
{\
    "http://example.com/vocab#parent" : [\
      {\
        "@id" : "#bart",\
        "http://example.com/vocab#name" : "Bart"\
      }, {\
        "@id" : "#lisa",\
        "http://example.com/vocab#name" : "Lisa"\
      }\
    ]\
  } :: ReverseValue .
print(result)
assert('result Bool: true' in result)       

### Node Object 
[JSON-LD 1.1 / Node Objects](https://www.w3.org/TR/json-ld11/#node-objects)

In [ ]:
%%maude
fmod NODE-OBJECT is
protecting KEYWORD .
protecting IDENTIFIER .
protecting JSON *( sort Object to Json-Object ) .

    

sorts Node-Object ^Node-Object .    
subsorts Node-Object ^Node-Object < Object .

sort Value .
    
var O : Object . var  N : Node-Object .
var V : Value .    
cmb O : ^Node-Object if
        ({ "@context"  : V  , ...} := O and not V :: Context         )    
     or ({ "@id"       : V  , ...} := O and not V :: Identifier      )
     or ({ "@graph"    : V  , ...} := O and not V :: Graph           )
     or ({ "@type "    : V  , ...} := O and not V :: Identifier      )
     or ({ "@reverse"  : V  , ...} := O and not V :: Reverse-Map     ) 
     or ({ "@included" : V  , ...} := O and not V :: Included-Block  )
     or ({ "@index"    : V  , ...} := O and not V :: String          )
     or ({ "@nest "    : V  , ...} := O and not V :: Property-Nesting) .

cmb O : Node-Object if not O :: ^Node-Object .
endfm     
    

### Expand

### Terms

### JSON-LD BASIC

In [ ]:
%%maude 
show modules .

### JSON-LD_TERM

### HTML

## TO-DO
* Remove trailibg newlines grom cell.
* A command that ads a path to MAUDE_LIB
* Implementar el poder acceder al Intérprete de maude
   Hacer un comando %maude shell de manera que haciendo la asignación
  ```python
  maude_shell = %maude shell
  ```
  tengamos acceso a la instancia del intérprete de la shell 

# Nbdev Export Survey
[nbdev](https://nbdev1.fast.ai/export.html#read_nb)

In [ ]:
from execnb.nbio import *
test_nb = read_nb('maude-magic.ipynb')
# print keys
test_nb.keys()
#Finding patterns
check_re('# export'